In [1]:
import pandas as pd
import sqlalchemy 
import psycopg2
import os
import numpy as np
import requests
import re
import copy
import sys
from pandas.api.types import is_numeric_dtype
from sklearn.linear_model import LinearRegression
from sqlalchemy import create_engine
import datetime
from datetime import datetime as dt
import geopandas as gpd
import topojson as tp
from shapely import geometry
sys.path.append('..')
from modules.transforms import *

In [2]:
DBname=os.environ['DB_NAME']
postgres_psswd=os.environ['POSTGRES_PASSWORD']
postgres_user=os.environ['POSTGRES_USER']
postgres_port=str(os.environ['POSTGRES_PORT'])

In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://'+postgres_user+':'+postgres_psswd+'@'+DBname+':'+postgres_port+'/superset')
# Create the connection
cnx = create_engine(postgres_str)

In [4]:
#guardar paso en csv
path='/data/ETLcache/'
now = dt.now()
timestamp = now.strftime("_%d%m%Y_%H%M%S")

In [5]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/historial_comunas.csv'

In [6]:
contagiados_comunal=pd.read_csv(url)
contagiados_comunal['Fecha']=pd.to_datetime(contagiados_comunal['Fecha'],errors='coerce').dt.to_pydatetime()

In [7]:
url='https://raw.githubusercontent.com/ivanMSC/COVID19_Chile/master/utils/comunasChile.csv'

In [8]:
info_comunal=pd.read_csv(url)

In [9]:
info_comunal=info_comunal.rename(columns={'CUT (Código Único Territorial)':'CUT'})

In [10]:
info_comunal['CUT'].astype(str).str[:-3].unique()

array(['15', '1', '2', '3', '4', '5', '6', '7', '16', '8', '9', '14',
       '10', '11', '12', '13'], dtype=object)

In [11]:
for k in ['@ivanMSC']:
    if not cnx.dialect.has_schema(cnx, k):
        print('schema '+k+' does not exist, creating it')
        cnx.execute(sqlalchemy.schema.CreateSchema(k))
    else:
        print('schema '+k+' exists, will not be created')

schema @ivanMSC does not exist, creating it


In [12]:
contagiados_comunal.set_index('Fecha',inplace=True)

In [13]:
contagiados_comunal['activos']=rolling(contagiados_comunal,numeric_col='Nuevos Confirmados',group_col='Comuna',window=14) 

In [14]:
contagiados_comunal=contagiados_comunal.reset_index()

In [15]:
contagiados_comunal=contagiados_comunal.drop('Region',axis=1)

In [16]:
contagiados_comunal=contagiados_comunal.merge(info_comunal[['CUT','Provincia','Region','Poblacion','Lat','Lon']],
                         on='CUT',how='left')

In [17]:
contagiados_comunal['Indicencia Acumulados']=contagiados_comunal['Confirmados Acumulados']/contagiados_comunal['Poblacion']*100000

In [18]:
contagiados_comunal['Indicencia Activos']=contagiados_comunal['activos']/contagiados_comunal['Poblacion']*100000

In [19]:
info_comunal

,CUT,Nombre,Provincia,Region,Superficie,Poblacion,Densidad,IDH 2005,IDH Categoria,Lat,Lon
0,15101,Arica,Arica,Arica y Parinacota,4799.4,247552.0,0.46,38.400,Alto,-18.45500,-70.29000
1,15102,Camarones,Arica,Arica y Parinacota,3927.0,1233.0,0.32,0.751,Alto,-19.01700,-69.86700
2,15201,Putre,Parinacota,Arica y Parinacota,5902.5,2515.0,0.47,0.707,Alto,-18.20000,-69.58300
3,15202,General Lagos,Parinacota,Arica y Parinacota,2244.4,810.0,0.31,0.670,Medio,-17.65300,-69.63500
4,1101,Iquique,Iquique,Tarapacá,2242.1,223463.0,85.40,0.766,Alto,-20.24400,-70.13900
...,...,...,...,...,...,...,...,...,...,...,...
343,13603,Isla de Maipo,Talagante,Metropolitana,189.0,40171.0,191.60,0.724,Alto,-33.75400,-70.88600
344,13604,Padre Hurtado,Talagante,Metropolitana,80.8,74188.0,782.80,0.728,Alto,-33.57600,-70.80000
345,13605,Peñaflor,Talagante,Metropolitana,69.0,101058.0,1307.30,0.753,Alto,-33.60600,-70.87700
346,5999,Barco Isabela Island,Valparaíso,Valparaíso,NaN,NaN,NaN,NaN,NaN,-33.01115,-71.59019


In [20]:
info_comunal['Codigo region']=info_comunal.apply(lambda x: int(str(x.CUT)[:-3]), axis=1)

In [21]:
name='contagios_comunal@ivanMSC'
schema='@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
contagiados_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
contagiados_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')
name='info_comunal@ivanMSC'
print("creating table "+name+' ,schema: '+schema)
info_comunal.to_sql(name, schema=schema,con=cnx,if_exists='replace')
print("saving table"+path+name+timestamp+'.csv in cache')
info_comunal.to_csv(path+name+timestamp+'.csv',encoding='utf-8')

creating table contagios_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/contagios_comunal@ivanMSC_06122021_011558.csv in cache
creating table info_comunal@ivanMSC ,schema: @ivanMSC
saving table/data/ETLcache/info_comunal@ivanMSC_06122021_011558.csv in cache


In [22]:
os.system('jupyter nbconvert --output /home/jovyan/work/ETLdocs/' + 'ETL_covid-chile.html' + ' --to html ' + 
          '/home/jovyan/work/ETL/@ivanMSC.ipynb')

[NbConvertApp] Converting notebook /home/jovyan/work/ETL/@ivanMSC.ipynb to html
[NbConvertApp] Writing 604660 bytes to /home/jovyan/work/ETLdocs/ETL_covid-chile.html


0